<a href="https://colab.research.google.com/github/Ujjwal10228/Projects/blob/main/Faker_News_Classifier_Using_LSTM_%26_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c fake-news

100% 46.5M/46.5M [00:03<00:00, 22.2MB/s]
100% 46.5M/46.5M [00:03<00:00, 15.1MB/s]


In [ ]:
#Extraction the compressed Dataset
from zipfile import ZipFile
dataset = '/content/fake-news.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The daataset is extracted


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
#Drop Nan Values
df = df.dropna()

In [ ]:
# Get Independent Features

X = df.drop('label', axis=1)

In [ ]:
# Get the Dependent features
y = df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.13.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense

In [ ]:
# VOcabulary Size
voc_size = 5000

**One_hot Representation**

In [ ]:
messages = X.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
onehot_repr = [one_hot(words, voc_size)for words in corpus]
onehot_repr

[[1724, 1804, 1299, 97, 1919, 2862, 699, 2211, 2638, 3662],
 [2838, 2696, 1448, 3251, 2047, 763, 2244],
 [1570, 2531, 3036, 2474],
 [3846, 2454, 1210, 1934, 423, 3600],
 [946, 2047, 4906, 1693, 2921, 4010, 2047, 3992, 3848, 2183],
 [2774,
  3902,
  887,
  2888,
  2489,
  4477,
  1231,
  4720,
  1494,
  661,
  2353,
  660,
  727,
  2370,
  2244],
 [1561, 2182, 513, 1862, 494, 2349, 4356, 2315, 1793, 4695, 887],
 [221, 2063, 4702, 3586, 4172, 4767, 4477, 2461, 1793, 4695, 887],
 [2234, 4135, 1625, 1337, 2131, 2531, 3038, 1966, 4477, 4426],
 [1532, 363, 4866, 4404, 4008, 210, 4377, 3948],
 [720, 3644, 3974, 4252, 3366, 1824, 1614, 923, 1299, 19, 965],
 [1934, 2907, 1919, 2531, 4477, 4172],
 [1084, 4819, 2538, 1630, 4686, 3306, 4393, 1193, 1465],
 [4934, 597, 1754, 2893, 2436, 2093, 4735, 1793, 4695, 887],
 [2631, 4441, 1079, 1439, 4827, 1793, 4695, 887],
 [4017, 4565, 3140, 2218, 4756, 2644, 1226, 3192, 548, 2577],
 [1446, 2324, 2696],
 [2988, 1178, 4774, 4749, 4477, 2861, 2712, 2244],
 [

**Embedding Representation**

In [ ]:
sent_length =20
embedded_docs=pad_sequences(onehot_repr, padding='pre', maxlen= sent_length) #Padding to make the sentences of equal lengths
print(embedded_docs)

[[   0    0    0 ... 2211 2638 3662]
 [   0    0    0 ... 2047  763 2244]
 [   0    0    0 ... 2531 3036 2474]
 ...
 [   0    0    0 ... 1793 4695  887]
 [   0    0    0 ... 2397 1047 3748]
 [   0    0    0 ... 4033 3013 1243]]


In [ ]:
len(embedded_docs)

18285

In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1724,
       1804, 1299,   97, 1919, 2862,  699, 2211, 2638, 3662], dtype=int32)

###Normal Model

In [ ]:
#Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


###Model With Bidirectional

In [ ]:
#Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
len(embedded_docs), y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 26s 84ms/step - loss: 0.3048 - accuracy: 0.8549 - val_loss: 0.1936 - val_accuracy: 0.9193
Epoch 2/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1311 - accuracy: 0.9487 - val_loss: 0.2120 - val_accuracy: 0.9170
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0950 - accuracy: 0.9663 - val_loss: 0.2156 - val_accuracy: 0.9180
Epoch 4/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0642 - accuracy: 0.9786 - val_loss: 0.2860 - val_accuracy: 0.9138
Epoch 5/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0457 - accuracy: 0.9861 - val_loss: 0.3456 - val_accuracy: 0.9115
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0341 - accuracy: 0.9903 - val_loss: 0.3586 - val_accuracy: 0.9171
Epoch 7/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0216 - accuracy: 0.9937 - val_loss: 0.4254 - val_accuracy: 0.913

In [ ]:
y_pred1=model1.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [ ]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 7ms/step


In [ ]:
from tensorflow.keras.layers import Dropout
#Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 51ms/step - loss: 0.3425 - accuracy: 0.8373 - val_loss: 0.2010 - val_accuracy: 0.9142
Epoch 2/10
192/192 [==============================] - 10s 53ms/step - loss: 0.1466 - accuracy: 0.9429 - val_loss: 0.1875 - val_accuracy: 0.9248
Epoch 3/10
192/192 [==============================] - 9s 49ms/step - loss: 0.1078 - accuracy: 0.9593 - val_loss: 0.2082 - val_accuracy: 0.9181
Epoch 4/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0864 - accuracy: 0.9693 - val_loss: 0.2130 - val_accuracy: 0.9213
Epoch 5/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0643 - accuracy: 0.9780 - val_loss: 0.2937 - val_accuracy: 0.9185
Epoch 6/10
192/192 [==============================] - 9s 49ms/step - loss: 0.0527 - accuracy: 0.9816 - val_loss: 0.3344 - val_accuracy: 0.9201
Epoch 7/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0364 - accuracy: 0.9874 - val_loss: 0.3450 - val_accuracy: 0.91

**Performance Metrics and Accuracy**

In [ ]:
y_pred=model.predict(X_test)

189/189 [==============================] - 2s 7ms/step


In [ ]:
y_test.shape

(6035,)

In [ ]:
## FOR BIDIRECTIONAL MODEL
from sklearn.metrics import confusion_matrix

y_pred1.reshape(6035,)
# y_pred.shape

threshold = 0.5
y_pred1_binary = np.where(y_pred1 > threshold, 1, 0)
confusion_matrix(y_test, y_pred1_binary)

array([[3179,  240],
       [ 345, 2271]])

In [ ]:
y_pred.reshape(6035,)
# y_pred.shape

threshold = 0.5
y_pred_binary = np.where(y_pred > threshold, 1, 0)
confusion_matrix(y_test, y_pred_binary)

array([[3085,  334],
       [ 180, 2436]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_binary)

0.9148301574150787

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred1_binary)

0.9030654515327258